In [6]:
import numpy as np
import rasterio as rio 
from datetime import datetime, timedelta
from affine import Affine
import xarray as xr
import pandas as pd

In [35]:
#give location of folder containing files from GEE
GEEpath='/nfs/attic/dfh/Aragon2/CSOdmn/WY/GEE_WY/';

#Info re: the files names out of Crumley GEE script
#give the 'root' pathname of the met files. Note: we will append things like _elev.tif,
# _prec.tif, and so on...

metfilename='cfsv2_2018-09-01_2019-10-01';

#give names of dem and land cover
demname='DEM_WY.tif';
lcname='NLCD2016_WY.tif';

#give name of domain (e.g., GOA, or Thompson_Pass, or something like that).
#This will only be used for option lat / lon grids.
domain='WY';

#give the desired output name of the met file
outfilename='mm_WY_2018-2020.dat'; #please use something descriptive to help identify the output file.

#give start time information
startyear=2018;
startmonth=9;
startday=1;
pointsperday=4; #use 4 for 6-hourly data, 8 for 3-hourly data, etc.
starthour=0;

# Landcover 

### NLCD LC Codes

|Code|NLCD2016 Landclass|Code|NLCD2016 Landclass|
| :-: | :-: | :-: | :-: |
|11 | open water |51 | dwarf shrub|
|12| ice / snow |52 | shrub/scrub|
|21 | developed; open space |71 | grassland/herbaceous|
|22 | developed; low intensity|72 | hedge/herbaceous|
|23 | developed; med intensity|73 | lichens|
|24 | developed; high intensity|74 | moss|
|31 | barren; rock, sand, clay|81 | pasture/hay|
|41 | deciduous forest|82 | cultivated crops|
|42 | evergreen forest|90 | woody wetlands|
|43 | mixed shrub|95 | emergent herbaceous wetlands|


### Snowmodel LC Codes

|Code  |Landcover Class |Code  |Landcover Class |
| --- | --- | --- | --- |
|1     | coniferous forest |13    | subalpine meadow  |      
|2     | deciduous forest |14    | tundra (non-tussock) |      
|3     | mixed forest |15    | tundra (tussock) |           
|4     | scattered short-conifer |16    | prostrate shrub tundra | 
|5     | clearcut conifer |17    | arctic gram. wetland |       
|6     | mesic upland shrub |18    | bare |       
|7     | xeric upland shrub |19    | water/possibly frozen |       
|8     | playa shrubland |20    | permanent snow/glacier |         
|9     | shrub wetland/riparian |21    | residential/urban |   
|10    | erect shrub tundra |22    | tall crops |       
|11    | low shrub tundra |23    | short crops |        
|12    | grassland rangeland  |24    | ocean |    

In [59]:
# landcover data 
INpath = GEEpath+lcname
def LC2SM(INpath,OUTpath):
    src=rio.open(INpath)
    lc=src.read(1)
    #ascii header 
    head = "ncols "+str(np.shape(dem)[1])+"\n\
    nrows "+str(np.shape(dem)[0])+"\n\
    xllcorner     "+str(int(src.bounds.left))+"\n\
    yllcorner     "+str(int(src.bounds.bottom))+"\n\
    cellsize      "+str(int(src.res[0]))+"\n\
    NODATA_value  -9999\n"

    #reassign lc from NLCD to SM classes
    DIR=DIR=np.empty([np.shape(lc)[0],np.shape(lc)[1]])
    DIR[lc == 11 ]=24
    DIR[lc == 12 ]=20
    DIR[lc == 21 ]=21
    DIR[lc == 22 ]=21
    DIR[lc == 23 ]=21
    DIR[lc == 24 ]=21
    DIR[lc == 31 ]=18
    DIR[lc == 41 ]=2
    DIR[lc == 42 ]=1
    DIR[lc == 43 ]=6
    DIR[lc == 51 ]=6
    DIR[lc == 52 ]=6
    DIR[lc == 71 ]=12
    DIR[lc == 72 ]=12
    DIR[lc == 73 ]=12
    DIR[lc == 74 ]=12
    DIR[lc == 81 ]=23
    DIR[lc == 82 ]=22
    DIR[lc == 90 ]=9
    DIR[lc == 95 ]=9
    DIR.astype(int)
    np.savetxt(OUTpath+'NLCD2016_'+domain+'.asc', DIR, fmt='%4.0f', header = head,comments='')

# DEM

In [55]:
# dem
INpath = GEEpath+demname
def DEM2SM(INpath, OUTpath):
    src=rio.open(INpath)
    dem=src.read(1)
    #ascii header 
    head = "ncols "+str(np.shape(dem)[1])+"\n\
    nrows "+str(np.shape(dem)[0])+"\n\
    xllcorner     "+str(int(src.bounds.left))+"\n\
    yllcorner     "+str(int(src.bounds.bottom))+"\n\
    cellsize      "+str(int(src.res[0]))+"\n\
    NODATA_value  -9999\n"
    np.savetxt(OUTpath+'DEM_'+domain+'.asc', dem, fmt='%4.0f', header = head,comments='')

# Met files 




### Emilio cfsv2 output

In [63]:
INpath = 'CFSv2_OperationalAnalysis_WY_40days.nc'
OUTpath = 'outmet.dat'#GEEpath+outfilename
def MET2SM(INpath, OUTpath):
    #compute number of grid points and time steps from size of 3d matrix
    data = xr.open_dataset('CFSv2_OperationalAnalysis_WY_40days.nc')
    
    Z = data['Geopotential_height_surface']

    PR = data['Precipitation_rate_surface_6_Hour_Average']

    H = data['Specific_humidity_height_above_ground']

    P = data['Pressure_surface']

    T = data['Temperature_height_above_ground']

    U = data['u-component_of_wind_height_above_ground']

    V = data['v-component_of_wind_height_above_ground']

    #compute number of grid points and time steps from size of 3d matrix
    t,y,x=PR.shape
    gridpts=x*y
    tsteps=t

    #create y m d h vectors
    year = pd.to_datetime(data.time.values).year
    month = pd.to_datetime(data.time.values).month
    day = pd.to_datetime(data.time.values).day
    hour = pd.to_datetime(data.time.values).hour
    
    #create ID numbers for the grid points
    ID=1000000+np.linspace(1,gridpts,gridpts)

    #create matrices of x and y values
    X, Y = np.meshgrid(data.easting.values, data.northing.values)
    X=X.flatten()
    Y=Y.flatten()

    #elevation is static (doesn't change with time)
    elev=Z[1,:,:].values.flatten()

    #find number of grid points with <0 elevation. Note: this is related to the
    #subroutine met_data_check in the preprocess_code.f. that subroutine seems
    #to suggest that negative elevations are ok (say, death valley). But, the
    #code itself checks for negative elevations and stops execution is any
    #negatives are found.
    I = np.where(elev>=0)
    validgridpts=np.shape(I)[1]

    #remove data at points with neg elevations
    ID=ID[I]
    X=X[I]
    Y=Y[I]
    elev=elev[I]

    #we are now ready to begin our main loop over the time steps.
    fid= open(OUTpath,"w+")

    for j in range(tsteps):
        #first we write the number of grid points
        fid.write('{0:6d}\n'.format(validgridpts))

        #prep data matrix for this time step. First, grab the jth time slice
        Prtmp=PR[j,:,:].values.flatten()
        Htmp=H[j,:,:].values.flatten()
        Ptmp=P[j,:,:].values.flatten()
        Ttmp=T[j,:,:].values.flatten()
        Utmp=U[j,:,:].values.flatten()
        Vtmp=V[j,:,:].values.flatten()

        #remove data at points with neg elevations
        Prtmp=Prtmp[I]
        Htmp=Htmp[I]
        Ptmp=Ptmp[I]
        Ttmp=Ttmp[I]
        Utmp=Utmp[I]
        Vtmp=Vtmp[I]


        #convert precip rate to precip DEPTH (mm) during time interval
        Prtmp=Prtmp*24*3600/pointsperday

        #convert specific hum. to RH from Clausius-Clapeyron. T is still in K
        RHtmp=0.263*Ptmp*Htmp*(np.exp(17.67*(Ttmp-273.16)/(Ttmp-29.65)))**(-1)

        #compute wind speed
        SPDtmp=np.sqrt(Utmp**2+Vtmp**2)

        #compute wind direction. 0-360, with 0 being true north! 90 east, etc.
        DIRtmp=np.arctan2(Utmp,Vtmp)
        K=np.where(DIRtmp>=180)
        J=np.where(DIRtmp<180)
        DIRtmp[K]=DIRtmp[K]+180
        DIRtmp[J]=DIRtmp[J]+180

        #put T in C
        Ttmp=Ttmp-273.16

        for z in range(len(Prtmp)): 

            fid.write('{0:5d}'.format(year[j])+'{0:3d}'.format(month[j])+
                      '{0:3d}'.format(day[j])+'{:6.3f}'.format(hour[j])+
                      '{0:9d}'.format(int(ID[z]))+'{:12.1f}'.format(X[z])+
                      '{:12.1f}'.format(Y[z])+'{:8.1f}'.format(elev[z])+
                      '{:9.2f}'.format(Ttmp[z])+'{:9.2f}'.format(RHtmp[z])+
                      '{:9.2f}'.format(SPDtmp[z])+'{:9.2f}'.format(DIRtmp[z])+
                      '{:9.2f}\n'.format(Prtmp[z]))
    fid.close()

In [64]:
MET2SM(INpath, OUTpath)

### GEE cfsv2 output

In [61]:
INpath = GEEpath+metfilename
OUTpath = GEEpath+outfilename
def MET2SM(INpath, OUTpath):
    #compute number of grid points and time steps from size of 3d matrix

    Z = xr.open_rasterio(INpath+'_elev.tif')

    PR = xr.open_rasterio(INpath+'_prec.tif')

    H = xr.open_rasterio(INpath+'_spechum.tif')

    P = xr.open_rasterio(INpath+'_surfpres.tif')

    T = xr.open_rasterio(INpath+'_tair.tif')

    U = xr.open_rasterio(INpath+'_uwind.tif')

    V = xr.open_rasterio(INpath+'_vwind.tif')

    #compute number of grid points and time steps from size of 3d matrix
    t,y,x=PR.shape
    gridpts=x*y
    tsteps=t

    #create y m d h vectors
    start_date =datetime.strptime(str(startyear)+'-'+str(startmonth)+'-'\
                                  +str(startday)+'-'+str(starthour),'%Y-%m-%d-%H')
    n = str(24/pointsperday)
    timestamp = pd.date_range(start_date, periods=tsteps, freq=n+'H')

    #create ID numbers for the grid points
    ID=1000000+np.linspace(1,gridpts,gridpts)

    #create matrices of x and y values
    transform = Affine.from_gdal(*PR.attrs['transform'])
    X, Y = np.meshgrid(np.arange(x)+0.5, np.arange(y)+0.5) * transform
    X=X.flatten()
    Y=Y.flatten()

    #elevation is static (doesn't change with time)
    elev=Z[1,:,:].values.flatten()

    #find number of grid points with <0 elevation. Note: this is related to the
    #subroutine met_data_check in the preprocess_code.f. that subroutine seems
    #to suggest that negative elevations are ok (say, death valley). But, the
    #code itself checks for negative elevations and stops execution is any
    #negatives are found.
    I = np.where(elev>=0)
    validgridpts=np.shape(I)[1]

    #remove data at points with neg elevations
    ID=ID[I]
    X=X[I]
    Y=Y[I]
    elev=elev[I]

    #we are now ready to begin our main loop over the time steps.
    fid= open(OUTpath,"w+")

    for j in range(tsteps):
        #first we write the number of grid points
        fid.write('{0:6d}\n'.format(validgridpts))

        #prep data matrix for this time step. First, grab the jth time slice
        Prtmp=PR[j,:,:].values.flatten()
        Htmp=H[j,:,:].values.flatten()
        Ptmp=P[j,:,:].values.flatten()
        Ttmp=T[j,:,:].values.flatten()
        Utmp=U[j,:,:].values.flatten()
        Vtmp=V[j,:,:].values.flatten()

        #remove data at points with neg elevations
        Prtmp=Prtmp[I]
        Htmp=Htmp[I]
        Ptmp=Ptmp[I]
        Ttmp=Ttmp[I]
        Utmp=Utmp[I]
        Vtmp=Vtmp[I]


        #convert precip rate to precip DEPTH (mm) during time interval
        Prtmp=Prtmp*24*3600/pointsperday

        #convert specific hum. to RH from Clausius-Clapeyron. T is still in K
        RHtmp=0.263*Ptmp*Htmp*(np.exp(17.67*(Ttmp-273.16)/(Ttmp-29.65)))**(-1)

        #compute wind speed
        SPDtmp=np.sqrt(Utmp**2+Vtmp**2)

        #compute wind direction. 0-360, with 0 being true north! 90 east, etc.
        DIRtmp=np.arctan2(Utmp,Vtmp)
        K=np.where(DIRtmp>=180)
        J=np.where(DIRtmp<180)
        DIRtmp[K]=DIRtmp[K]+180
        DIRtmp[J]=DIRtmp[J]+180

        #put T in C
        Ttmp=Ttmp-273.16

        for z in range(len(Prtmp)):

            fid.write('{0:5d}'.format(timestamp[j].year)+'{0:3d}'.format(timestamp[j].month)+
                      '{0:3d}'.format(timestamp[j].day)+'{:6.3f}'.format(timestamp[j].hour)+
                      '{0:9d}'.format(int(ID[z]))+'{:12.1f}'.format(X[z])+
                      '{:12.1f}'.format(Y[z])+'{:8.1f}'.format(elev[z])+
                      '{:9.2f}'.format(Ttmp[z])+'{:9.2f}'.format(RHtmp[z])+
                      '{:9.2f}'.format(SPDtmp[z])+'{:9.2f}'.format(DIRtmp[z])+
                      '{:9.2f}\n'.format(Prtmp[z]))
    fid.close()

In [62]:
MET2SM(INpath, 'outmet.dat')

/nfs/attic/dfh/miniconda/envs/snowmodelcal/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: Right multiplication will be prohibited in version 3.0


# Extramet 
Lat lon grids for large domain 

In [ ]:
%% In this section we write out the optional files containing lat / lon values.
% Either grads or ascii is acceptable. I prefer ascii...

if FLAG_DEM_EXTRA
   disp('Converting projected coords to lat / lon')
   disp(' ')
    %load file.
    [DEM,R_dem]=geotiffread(fullfile(pathname,demname));
    %get projection info
    proj=geotiffinfo(fullfile(pathname,demname));
    %convert projection info to mapping structure
    mstruct=geotiff2mstruct(proj);

    %create matrices and vectors of x and y values
    info=geotiffinfo(fullfile(pathname,demname));
    [X,Y]=pixcenters(info,'makegrid');
    x=X(1,:);
    y=Y(:,1);

    %convert coords from projected to geographic
    [LAT,LON]=minvtran(mstruct,X,Y);

    %write out files
    arcgridwrite(fullfile(pathname,'grid_lat.asc'),x,y,LAT,'grid_mapping','center','precision',5);
    arcgridwrite(fullfile(pathname,'_grid_lon.asc'),x,y,LON,'grid_mapping','center','precision',5);
%     arcgridwrite(fullfile(pathname,[domain 'grid_lat.asc']),x,y,LAT,'grid_mapping','center','precision',5);
%     arcgridwrite(fullfile(pathname,[domain '_grid_lon.asc']),x,y,LON,'grid_mapping','center','precision',5);
end


In [ ]:
def LR2SM():
    

In [ ]:
        #compute wind direction. 0-360, with 0 being true north! 90 east, etc.
        DIRtmp=np.arctan2(Vtmp,Utmp)* 180 / np.pi
        J=np.where((DIRtmp>=0) & (DIRtmp<=90))
        K=np.where(DIRtmp>90)
        L=np.where((DIRtmp<0) & (DIRtmp>=-90))
        M=np.where(DIRtmp<-90)
        DIRtmp[J]=90-DIRtmp[J]+180
        DIRtmp[K]=270 - DIRtmp[K]
        DIRtmp[L]=np.abs(90+ DIRtmp[L])
        DIRtmp[M]=270+np.abs(DIRtmp[M])